In [1]:
%pdb

Automatic pdb calling has been turned ON


In [2]:
import math
from sortedcontainers import SortedDict
import random
import numpy 
import matplotlib.pyplot as plt

In [3]:
# Code for generating positions, distances, travel times and preferences
def generate_positions(males, x_dim, y_dim):
    Xs = numpy.random.rand(males) * x_dim
    Ys = numpy.random.rand(males) * y_dim
    return [Xs,Ys]

def compute_distances_travel_times(males, positions, bird_speed):
    male_dist = numpy.zeros((males, males))
    travel_times = numpy.zeros((males, males))
    for i in range(males):
        for j in range(i + 1, males):
            dist = math.sqrt((positions[0][j] - positions[0][i]) ** 2 + (positions[1][j] - positions[1][i]) ** 2)
            travel = dist / bird_speed
            male_dist[j][i] = dist
            male_dist[i][j] = dist
            travel_times[j][i] = travel
            travel_times[i][j] = travel
    return (male_dist, travel_times)

def compute_visit_preferences(males, distances, lambda_dist):
    # compute exponential of each coefficient
    visit_preferences = numpy.exp(-lambda_dist * distances)
    # remove the identity matrix (exp(0) = 1)
    visit_preferences = visit_preferences - numpy.eye(males)
    # make rows sum to one
    visit_preferences = (visit_preferences.transpose() / numpy.sum(visit_preferences, 1)).transpose()
    return visit_preferences

In [5]:
# functions to generate tickets and manage timeline
def generate_ticket(start_time, end_time, length_activity, owner, action, target):
    global timeline
    ticket = {"start_time": start_time,
              "end_time": end_time,
              "length_activity": length_activity,
              "owner": owner,
              "action": action,
              "target": target
             }
    # now add to timeline
    timeline[(ticket["end_time"], ticket["owner"])] = ticket


In [6]:
# ACTION FUNCTIONS
# Each action generates a ticket, and updates the state of the owner (and possibly the target)

def draw_foraging_time(start_time):
    # note -- for now take the absolute value. Can be changed
    # note: HARD CODED PARAMETERS!
    next_time = numpy.abs(numpy.random.normal(loc = 0.4, scale = 0.167))
    return next_time + start_time

def action_forage(bird_id, current_time):
    global birds
    # generate the time it takes to forage
    # note: HARD CODED PARAMETERS!
    time_spent_foraging = numpy.random.gamma(shape = 1.5, scale = 5.0) / 60.0
    time_action_ends = current_time + time_spent_foraging
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_foraging,
                   owner = bird_id,
                   action = "foraging",
                   target = -1)
    # update the bird:
    birds[bird_id]["current_state"] = "foraging"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends
    # update the time to next foraging: start counting when foraging ended
    birds[bird_id]["next_foraging_time"] = draw_foraging_time(time_action_ends)

def action_stay_at_bower(bird_id, current_time):
    global birds
    # generate the length of the stay
    # notes: Taking abs; HARD CODED PARAMS!
    time_spent_at_bower = numpy.abs(numpy.random.normal(loc = 0.1583, scale= 0.09755))
    time_action_ends = current_time + time_spent_at_bower
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_at_bower,
                   owner = bird_id,
                   action = "staying at bower",
                   target = -1)
    # update the bird:
    birds[bird_id]["current_state"] = "staying at bower"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends

def action_travel_to_maraud(bird_id, current_time):
    global birds
    # choose who to maraud
    tmp = numpy.random.rand()
    target = numpy.argwhere(birds[bird_id]["travel_preferences"] > tmp)[0][0]
    time_to_travel = birds[bird_id]["travel_times"][target]
    time_action_ends = current_time + time_to_travel
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_to_travel,
                   owner = bird_id,
                   action = "travel to maraud",
                   target = target)
    # update the bird:
    birds[bird_id]["current_state"] = "travel to maraud"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends

def action_repair_bower(bird_id, current_time):
    global birds
    # generate the length of the repair bout
    # notes: Taking abs; HARD CODED PARAMS!
    time_spent_repairing_bower = numpy.abs(numpy.random.normal(loc = 0.1583, scale = 0.09755))
    time_action_ends = current_time + time_spent_repairing_bower
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_repairing_bower,
                   owner = bird_id,
                   action = "repairing bower",
                   target = -1)
    # update the bird:
    birds[bird_id]["current_state"] = "repairing bower"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends
    # note: already accounts for the improvements
    birds[bird_id]["bower_state"] = birds[bird_id]["bower_state"] + time_spent_repairing_bower
    # cannot make it better than 0
    if birds[bird_id]["bower_state"] > 0.0:
        birds[bird_id]["bower_state"] = 0.0

def action_maraud(marauder_id, marauder_target, current_time):
    global birds
    # note: HARD CODED PARAMS!
    time_spent_marauding = 0.1 
    # note: HARD CODED PARAMS!
    damage_to_bower = 6.0 
    time_action_ends = current_time + time_spent_marauding
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_marauding,
                   owner = marauder_id,
                   action = "marauding",
                   target = marauder_target)
    # update marauder
    birds[marauder_id]["current_state"] = "marauding"
    birds[marauder_id]["action_starts"] = current_time
    birds[marauder_id]["action_ends"] = time_action_ends
    # update target
    birds[marauder_target]["bower_state"] = birds[marauder_target]["bower_state"] - damage_to_bower
    

def action_travel_from_maraud(marauder_id, marauder_target, current_time):
    global birds
    time_from_travel = birds[marauder_id]["travel_times"][marauder_target]
    time_action_ends = current_time + time_from_travel
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_from_travel,
                   owner = marauder_id,
                   action = "travel from maraud",
                   target = marauder_target)
    # update the bird:
    birds[marauder_id]["current_state"] = "travel from maraud"
    birds[marauder_id]["action_starts"] = current_time
    birds[marauder_id]["action_ends"] = time_action_ends

In [7]:
# this function should be called every time the bird 
# 1) is back to the bower (from foraging, marauding), 
# 2) has finished repairing the bower, 
# 3) or has finished a stint at staying at bower
def choose_action(bird, current_time):
    global t_max
    # stop generating actions at t_max
    if current_time < t_max:
        # if it's time to eat
        if current_time > bird["next_foraging_time"]:
            action_forage(bird["id"], current_time)
        # if the bower needs repair
        elif bird["bower_state"] < 0.0:
            # go repair
            action_repair_bower(bird["id"], current_time)
        # check if it wants to maraud
        elif numpy.random.rand() < bird["probability_maraud"]:
            # go maraud
            action_travel_to_maraud(bird["id"], current_time)
        else:
            # stay at bower
            action_stay_at_bower(bird["id"], current_time)
       

In [8]:
def initialize_bird(bird_id, bird_strategy, bird_xy, bird_preferences, bird_travel_times):
    # initialize dictionary
    bird = {"id": bird_id,
            "current_state": "none",
            "action_starts": 0.0,
            "action_ends": -1.0,
            "probability_maraud": bird_strategy,
            "bower_state": 0.0,
            "successful_mating": 0,
            "next_foraging_time": draw_foraging_time(0.0),
            "travel_preferences": numpy.cumsum(bird_preferences), # note: store cumulative probability for faster choice
            "travel_times": bird_travel_times, 
            "position": bird_xy
            }
    return(bird)

In [9]:
# this is the most important function!
def read_ticket(tic):
    global birds
    if tic["action"] in ("foraging", "staying at bower", "repairing bower", "travel from maraud"):
        # I am back at the bower, choose new action
        choose_action(birds[tic["owner"]], tic["end_time"])
    elif tic["action"] == "travel to maraud":
        # check whether the target is at home
        go_maraud = True
        if birds[tic["target"]]["current_state"] in ("staying at bower", "repairing bower"):
            go_maraud = False
        if birds[tic["target"]]["bower_state"] < 0.0:
            go_maraud = False
        if go_maraud: # maraud
            action_maraud(tic["owner"], tic["target"], tic["end_time"])
        else: # go back
            action_travel_from_maraud(tic["owner"], tic["target"], tic["end_time"])
    elif tic["action"] == "marauding":
        # travel back
        action_travel_from_maraud(tic["owner"], tic["target"], tic["end_time"])
    else:
        1 / 0 # something went horribly wrong
    

In [10]:
# Global Variables
# MALES
males = 10 # number of male birds
# FEMALES
females = 5 # number of female birds
# POSITIONS AND TRAVEL TIME
x_dim, y_dim = 400, 500 # dimensions of environment
bird_speed = 12 * 3600 # m/hr (12 m/s)
# now choose lambda_dist, controlling the probability of traveling to a neighbor
# the probability of choosing a neighbor at distance x is proportional to exp(-\lambda x)
# choose lambda such that 99% of the mass is before 800 meters
improb = 0.99
improb_distance = 800
lambda_dist = - math.log(1.0 - improb) / improb_distance
# TIMELINE
timeline = SortedDict()
t_max = 100 #12 * 30 # time when simulation ends
# BIRDS
birds = []
strategies = numpy.random.random(males)
# DEBUG
past_events = []
timeline_lengths = []

######## CODE SIMULATION #########
numpy.random.seed(0)

# initialize positions, travel times and preferences
positions = generate_positions(males, x_dim, y_dim)
distances, travel_times = compute_distances_travel_times(males, positions, bird_speed)
visit_preferences = compute_visit_preferences(males, distances, lambda_dist)
for i in range(males):
    birds.append(initialize_bird(i, 
                                 strategies[i], 
                                 (positions[0][i], positions[1][i]), 
                                 visit_preferences[i],
                                 travel_times[i]))
    # choose its first action
    choose_action(birds[-1], 0.0)

# this is the main loop
while len(timeline) > 0:
    current_ticket = timeline.popitem(0)
    read_ticket(current_ticket[1])
    # for debug: store all the past tickets
    past_events.append(current_ticket)
    # and every so often check the length of the timeline
    if numpy.random.rand() < 0.01:
        timeline_lengths.append(len(timeline))


In [11]:
# DEBUG: check that timeline does not bloat!
print(timeline_lengths)

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


In [12]:
# DEBUG: MAKE SURE THAT TIMELINE MAKES SENSE
for my_bird in range(males):
    print("#################################")
    print("TIMELINE OF BIRD", my_bird)
    print("#################################")
    for event in past_events:
        tic = event[1]
        if tic["owner"] == my_bird:
            print(round(tic["start_time"], 3), "-", round(tic["end_time"], 3),
                 "->", tic["action"], "[target:", tic["target"], "]")
    print("#################################")
    print("")
    print("")
    

#################################
TIMELINE OF BIRD 0
#################################
0.0 - 0.003 -> travel to maraud [target: 7 ]
0.003 - 0.006 -> travel from maraud [target: 7 ]
0.006 - 0.009 -> travel to maraud [target: 2 ]
0.009 - 0.012 -> travel from maraud [target: 2 ]
0.012 - 0.014 -> travel to maraud [target: 2 ]
0.014 - 0.017 -> travel from maraud [target: 2 ]
0.017 - 0.019 -> travel to maraud [target: 9 ]
0.019 - 0.021 -> travel from maraud [target: 9 ]
0.021 - 0.022 -> travel to maraud [target: 9 ]
0.022 - 0.024 -> travel from maraud [target: 9 ]
0.024 - 0.026 -> travel to maraud [target: 3 ]
0.026 - 0.027 -> travel from maraud [target: 3 ]
0.027 - 0.029 -> travel to maraud [target: 3 ]
0.029 - 0.03 -> travel from maraud [target: 3 ]
0.03 - 0.031 -> staying at bower [target: -1 ]
0.031 - 0.034 -> travel to maraud [target: 7 ]
0.034 - 0.037 -> travel from maraud [target: 7 ]
0.037 - 0.241 -> staying at bower [target: -1 ]
0.241 - 0.244 -> travel to maraud [target: 2 ]
0.244 

20.968 - 20.971 -> travel to maraud [target: 7 ]
20.971 - 20.974 -> travel from maraud [target: 7 ]
20.974 - 20.976 -> travel to maraud [target: 3 ]
20.976 - 20.977 -> travel from maraud [target: 3 ]
20.977 - 21.331 -> staying at bower [target: -1 ]
21.331 - 21.552 -> foraging [target: -1 ]
21.552 - 21.554 -> travel to maraud [target: 3 ]
21.554 - 21.555 -> travel from maraud [target: 3 ]
21.555 - 21.775 -> staying at bower [target: -1 ]
21.775 - 21.777 -> travel to maraud [target: 9 ]
21.777 - 21.779 -> travel from maraud [target: 9 ]
21.779 - 22.027 -> staying at bower [target: -1 ]
22.027 - 22.03 -> travel to maraud [target: 2 ]
22.03 - 22.033 -> travel from maraud [target: 2 ]
22.033 - 22.281 -> staying at bower [target: -1 ]
22.281 - 22.356 -> foraging [target: -1 ]
22.356 - 22.359 -> travel to maraud [target: 1 ]
22.359 - 22.362 -> travel from maraud [target: 1 ]
22.362 - 22.366 -> travel to maraud [target: 7 ]
22.366 - 22.369 -> travel from maraud [target: 7 ]
22.369 - 22.371 ->

39.434 - 39.499 -> repairing bower [target: -1 ]
39.499 - 39.717 -> repairing bower [target: -1 ]
39.717 - 39.818 -> repairing bower [target: -1 ]
39.818 - 39.915 -> foraging [target: -1 ]
39.915 - 39.917 -> travel to maraud [target: 9 ]
39.917 - 39.919 -> travel from maraud [target: 9 ]
39.919 - 39.927 -> travel to maraud [target: 5 ]
39.927 - 39.935 -> travel from maraud [target: 5 ]
39.935 - 39.943 -> travel to maraud [target: 5 ]
39.943 - 39.952 -> travel from maraud [target: 5 ]
39.952 - 39.955 -> travel to maraud [target: 8 ]
39.955 - 40.055 -> marauding [target: 8 ]
40.055 - 40.059 -> travel from maraud [target: 8 ]
40.059 - 40.225 -> staying at bower [target: -1 ]
40.225 - 40.226 -> travel to maraud [target: 9 ]
40.226 - 40.228 -> travel from maraud [target: 9 ]
40.228 - 40.232 -> travel to maraud [target: 8 ]
40.232 - 40.236 -> travel from maraud [target: 8 ]
40.236 - 40.237 -> travel to maraud [target: 3 ]
40.237 - 40.239 -> travel from maraud [target: 3 ]
40.239 - 40.242 -> 

58.34 - 58.342 -> travel from maraud [target: 3 ]
58.342 - 58.444 -> staying at bower [target: -1 ]
58.444 - 58.538 -> staying at bower [target: -1 ]
58.538 - 58.698 -> staying at bower [target: -1 ]
58.698 - 58.792 -> foraging [target: -1 ]
58.792 - 58.897 -> repairing bower [target: -1 ]
58.897 - 59.156 -> repairing bower [target: -1 ]
59.156 - 59.23 -> foraging [target: -1 ]
59.23 - 59.238 -> repairing bower [target: -1 ]
59.238 - 59.245 -> repairing bower [target: -1 ]
59.245 - 59.294 -> repairing bower [target: -1 ]
59.294 - 59.515 -> repairing bower [target: -1 ]
59.515 - 59.711 -> repairing bower [target: -1 ]
59.711 - 59.81 -> repairing bower [target: -1 ]
59.81 - 59.922 -> repairing bower [target: -1 ]
59.922 - 60.069 -> repairing bower [target: -1 ]
60.069 - 60.119 -> foraging [target: -1 ]
60.119 - 60.311 -> repairing bower [target: -1 ]
60.311 - 60.416 -> repairing bower [target: -1 ]
60.416 - 60.498 -> repairing bower [target: -1 ]
60.498 - 60.542 -> repairing bower [targe

83.98 - 84.043 -> staying at bower [target: -1 ]
84.043 - 84.06 -> staying at bower [target: -1 ]
84.06 - 84.063 -> travel to maraud [target: 7 ]
84.063 - 84.067 -> travel from maraud [target: 7 ]
84.067 - 84.075 -> travel to maraud [target: 5 ]
84.075 - 84.083 -> travel from maraud [target: 5 ]
84.083 - 84.085 -> travel to maraud [target: 9 ]
84.085 - 84.087 -> travel from maraud [target: 9 ]
84.087 - 84.09 -> travel to maraud [target: 1 ]
84.09 - 84.093 -> travel from maraud [target: 1 ]
84.093 - 84.097 -> travel to maraud [target: 8 ]
84.097 - 84.101 -> travel from maraud [target: 8 ]
84.101 - 84.229 -> staying at bower [target: -1 ]
84.229 - 84.232 -> travel to maraud [target: 1 ]
84.232 - 84.235 -> travel from maraud [target: 1 ]
84.235 - 84.239 -> travel to maraud [target: 1 ]
84.239 - 84.242 -> travel from maraud [target: 1 ]
84.242 - 84.319 -> staying at bower [target: -1 ]
84.319 - 84.323 -> travel to maraud [target: 1 ]
84.323 - 84.326 -> travel from maraud [target: 1 ]
84.32

...

10.984 - 11.122 -> foraging [target: -1 ]
11.122 - 11.131 -> travel to maraud [target: 5 ]
11.131 - 11.139 -> travel from maraud [target: 5 ]
11.139 - 11.143 -> travel to maraud [target: 3 ]
11.143 - 11.148 -> travel from maraud [target: 3 ]
11.148 - 11.152 -> travel to maraud [target: 3 ]
11.152 - 11.156 -> travel from maraud [target: 3 ]
11.156 - 11.249 -> staying at bower [target: -1 ]
11.249 - 11.25 -> travel to maraud [target: 7 ]
11.25 - 11.25 -> travel from maraud [target: 7 ]
11.25 - 11.251 -> travel to maraud [target: 7 ]
11.251 - 11.252 -> travel from maraud [target: 7 ]
11.252 - 11.258 -> travel to maraud [target: 9 ]
11.258 - 11.263 -> travel from maraud [target: 9 ]
11.263 - 11.417 -> staying at bower [target: -1 ]
11.417 - 11.421 -> travel to maraud [target: 2 ]
11.421 - 11.425 -> travel from maraud [target: 2 ]
11.425 - 11.508 -> staying at bower [target: -1 ]
11.508 - 11.742 -> foraging [target: -1 ]
11.742 - 11.743 -> travel to maraud [target: 7 ]
11.743 - 11.744 -> tr

38.444 - 38.481 -> foraging [target: -1 ]
38.481 - 38.485 -> travel to maraud [target: 3 ]
38.485 - 38.49 -> travel from maraud [target: 3 ]
38.49 - 38.491 -> travel to maraud [target: 7 ]
38.491 - 38.492 -> travel from maraud [target: 7 ]
38.492 - 38.495 -> travel to maraud [target: 1 ]
38.495 - 38.499 -> travel from maraud [target: 1 ]
38.499 - 38.509 -> staying at bower [target: -1 ]
38.509 - 38.519 -> travel to maraud [target: 4 ]
38.519 - 38.528 -> travel from maraud [target: 4 ]
38.528 - 38.533 -> travel to maraud [target: 3 ]
38.533 - 38.537 -> travel from maraud [target: 3 ]
38.537 - 38.603 -> staying at bower [target: -1 ]
38.603 - 38.691 -> foraging [target: -1 ]
38.691 - 38.692 -> travel to maraud [target: 7 ]
38.692 - 38.693 -> travel from maraud [target: 7 ]
38.693 - 38.703 -> travel to maraud [target: 6 ]
38.703 - 38.713 -> travel from maraud [target: 6 ]
38.713 - 38.714 -> travel to maraud [target: 7 ]
38.714 - 38.715 -> travel from maraud [target: 7 ]
38.715 - 38.72 -> 

73.405 - 73.406 -> travel from maraud [target: 7 ]
73.406 - 73.407 -> travel to maraud [target: 7 ]
73.407 - 73.408 -> travel from maraud [target: 7 ]
73.408 - 73.409 -> travel to maraud [target: 7 ]
73.409 - 73.41 -> travel from maraud [target: 7 ]
73.41 - 73.414 -> travel to maraud [target: 3 ]
73.414 - 73.418 -> travel from maraud [target: 3 ]
73.418 - 73.424 -> travel to maraud [target: 9 ]
73.424 - 73.429 -> travel from maraud [target: 9 ]
73.429 - 73.43 -> travel to maraud [target: 7 ]
73.43 - 73.431 -> travel from maraud [target: 7 ]
73.431 - 73.464 -> staying at bower [target: -1 ]
73.464 - 73.465 -> travel to maraud [target: 7 ]
73.465 - 73.466 -> travel from maraud [target: 7 ]
73.466 - 73.65 -> staying at bower [target: -1 ]
73.65 - 73.791 -> foraging [target: -1 ]
73.791 - 74.009 -> repairing bower [target: -1 ]
74.009 - 74.392 -> repairing bower [target: -1 ]
74.392 - 74.442 -> foraging [target: -1 ]
74.442 - 74.744 -> repairing bower [target: -1 ]
74.744 - 74.745 -> repai

23.429 - 23.431 -> travel to maraud [target: 3 ]
23.431 - 23.432 -> travel from maraud [target: 3 ]
23.432 - 23.442 -> staying at bower [target: -1 ]
23.442 - 23.443 -> travel to maraud [target: 0 ]
23.443 - 23.445 -> travel from maraud [target: 0 ]
23.445 - 23.45 -> travel to maraud [target: 1 ]
23.45 - 23.455 -> travel from maraud [target: 1 ]
23.455 - 23.615 -> staying at bower [target: -1 ]
23.615 - 23.707 -> foraging [target: -1 ]
23.707 - 23.712 -> travel to maraud [target: 2 ]
23.712 - 23.716 -> travel from maraud [target: 2 ]
23.716 - 23.721 -> travel to maraud [target: 1 ]
23.721 - 23.821 -> marauding [target: 1 ]
23.821 - 23.826 -> travel from maraud [target: 1 ]
23.826 - 23.827 -> travel to maraud [target: 3 ]
23.827 - 23.829 -> travel from maraud [target: 3 ]
23.829 - 23.833 -> travel to maraud [target: 2 ]
23.833 - 23.837 -> travel from maraud [target: 2 ]
23.837 - 23.839 -> travel to maraud [target: 0 ]
23.839 - 23.939 -> marauding [target: 0 ]
23.939 - 23.94 -> travel fr

59.871 - 60.018 -> staying at bower [target: -1 ]
60.018 - 60.02 -> travel to maraud [target: 0 ]
60.02 - 60.022 -> travel from maraud [target: 0 ]
60.022 - 60.024 -> travel to maraud [target: 0 ]
60.024 - 60.026 -> travel from maraud [target: 0 ]
60.026 - 60.031 -> travel to maraud [target: 8 ]
60.031 - 60.037 -> travel from maraud [target: 8 ]
60.037 - 60.041 -> travel to maraud [target: 7 ]
60.041 - 60.046 -> travel from maraud [target: 7 ]
60.046 - 60.129 -> staying at bower [target: -1 ]
60.129 - 60.137 -> staying at bower [target: -1 ]
60.137 - 60.199 -> staying at bower [target: -1 ]
60.199 - 60.204 -> travel to maraud [target: 7 ]
60.204 - 60.209 -> travel from maraud [target: 7 ]
60.209 - 60.214 -> travel to maraud [target: 8 ]
60.214 - 60.22 -> travel from maraud [target: 8 ]
60.22 - 60.333 -> staying at bower [target: -1 ]
60.333 - 60.594 -> foraging [target: -1 ]
60.594 - 60.598 -> travel to maraud [target: 2 ]
60.598 - 60.602 -> travel from maraud [target: 2 ]
60.602 - 60.